In [161]:
import pandas as pd
from math import sqrt
import numpy as np

In [162]:
amazon_df=pd.read_csv(r"D:\Gorian\Data science basics\Portfolio\amazon_products.csv", low_memory=False)

In [163]:
amazon_df.shape

(34660, 21)

#### Data preprocessing

In [164]:
amazon_df.columns

Index(['id', 'name', 'asins', 'brand', 'categories', 'keys', 'manufacturer',
       'reviews.date', 'reviews.dateAdded', 'reviews.dateSeen',
       'reviews.didPurchase', 'reviews.doRecommend', 'reviews.id',
       'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.userCity',
       'reviews.userProvince', 'reviews.username'],
      dtype='object')

In [165]:
data=amazon_df[['id', 'name', 'reviews.rating', 'reviews.username']]

In [166]:
#Checking percentage of missing values
data.isnull().sum()/len(data)*100

id                   0.000000
name                19.503751
reviews.rating       0.095211
reviews.username     0.005770
dtype: float64

In [167]:
data['name'].fillna(amazon_df['categories'], inplace=True)

In [168]:
data.dropna(subset=['reviews.rating', 'reviews.username'], inplace=True)

C:\Users\Stas\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [169]:
data.head()

,id,name,reviews.rating,reviews.username
0,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",5.0,Adapter
1,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",5.0,truman
2,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",5.0,DaveZ
3,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",4.0,Shacks
4,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",5.0,explore42


In [171]:
new = data[data['reviews.username'].isin(selected['reviews.username'].tolist())]
new.head()

,id,name,reviews.rating,reviews.username
16,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",4.0,Linda
26,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",4.0,Dave
27,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",5.0,James
34,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",5.0,rick
36,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",4.0,Gift


In [172]:
new.rename(columns={'id':'ProductId','reviews.username': 'UserID', 'reviews.rating': 'rating'}, inplace=True)

C:\Users\Stas\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [173]:
new

,ProductId,name,rating,UserID
16,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",4.0,Linda
26,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",4.0,Dave
27,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",5.0,James
34,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",5.0,rick
36,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",4.0,Gift
...,...,...,...,...
34623,AV1YE_muvKc47QAVgpwE,"Back To College,College Electronics,College Tv...",4.0,Jack
34635,AVpf_znpilAPnD_xlvAF,"Electronics,Amazon Device Accessories,Kindle S...",5.0,David
34642,AVpggqsrLJeJML43O5zp,"Power Adapters & Cables,Electronics,USB Cables",5.0,Amazon Customer
34653,AVpfiBlyLJeJML43-4Tp,"Computers/Tablets & Networking,Tablet & eBook ...",1.0,Scooter


### Collaborative filtering

##### User to recommend to

In [174]:
userInput = [
            {'ProductId':'AVpfl8cLLJeJML43AE3S', 'rating':5},
            {'ProductId':'AVpfpK8KLJeJML43BCuD', 'rating':4},
            {'ProductId':'AV1YE_muvKc47QAVgpwE', 'rating':3},
            {'ProductId':"AVqkIhwDv8e3D1O-lebb", 'rating':5},
            {'ProductId':'AV1YnRtnglJLPUi8IJmV', 'rating':4}
         ] 
inputProducts = pd.DataFrame(userInput)
inputProducts

,ProductId,rating
0,AVpfl8cLLJeJML43AE3S,5
1,AVpfpK8KLJeJML43BCuD,4
2,AV1YE_muvKc47QAVgpwE,3
3,AVqkIhwDv8e3D1O-lebb,5
4,AV1YnRtnglJLPUi8IJmV,4


###### Defining users that has bought and rated the same products

In [175]:
userSubset = new[new['ProductId'].isin(inputProducts['ProductId'].tolist())]
userSubset.head()

,ProductId,name,rating,UserID
16,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",4.0,Linda
26,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",4.0,Dave
27,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",5.0,James
34,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",5.0,rick
36,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",4.0,Gift


In [176]:
userSubsetGroup = userSubset.groupby(['UserID'])

In [177]:
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

In [178]:
userSubsetGroup[0:3]

[('Mike',
                    ProductId  \
  73     AVqkIhwDv8e3D1O-lebb   
  182    AVqkIhwDv8e3D1O-lebb   
  311    AVqkIhwDv8e3D1O-lebb   
  706    AVqkIhwDv8e3D1O-lebb   
  758    AVqkIhwDv8e3D1O-lebb   
  ...                     ...   
  33162  AV1YE_muvKc47QAVgpwE   
  33546  AV1YE_muvKc47QAVgpwE   
  33756  AV1YE_muvKc47QAVgpwE   
  34239  AV1YE_muvKc47QAVgpwE   
  34450  AV1YE_muvKc47QAVgpwE   
  
                                                      name  rating UserID  
  73     All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...     4.0   Mike  
  182    All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...     5.0   Mike  
  311    All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...     5.0   Mike  
  706    All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...     3.0   Mike  
  758    All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...     4.0   Mike  
  ...                                                  ...     ...    ...  
  33162  Back To College,College Electronics,College Tv... 

#### Similarity of users' tastes

In [191]:
userSubsetGroup = userSubsetGroup[0:100]
#Storing the Pearson Correlation in a dictionary, where the key is the UserID and the value is the similarity coefficient
pearsonCorrelationDict = {}

#For every user group in the subset:
for name, group in userSubsetGroup:
    
    group = group.sort_values(by='ProductId')
    inputProducts = inputProducts.sort_values(by='ProductId')
    
    nRatings = len(group)
 
    temp_df = inputProducts[inputProducts['ProductId'].isin(group['ProductId'].tolist())]

    tempRatingList = temp_df['rating'].tolist()
    
    tempGroupList = group['rating'].tolist()
    #calculating the pearson correlation between two users x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else - meaning 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

In [192]:
pearsonCorrelationDict.items()

dict_items([('Mike', -0.06016285526932847), ('John', 0.1434755917412464), ('Chris', -0.08581045965096305), ('Bill', 0.18713989777774015), ('Dave', 0.006327593294407015), ('Tony', 0.19905363468548964), ('Rick', -0.1228756718653266), ('Steve', 0.00881922003268491), ('Matt', -0.06271661239095518), ('Nick', 0.06536011210576019), ('David', 0.25249690660420865), ('Kevin', -0.07283972936813027), ('Mark', 0.13625956071907355), ('Rich', 0.13885762993121947), ('Anonymous', 0.21120377193190057), ('Robert', 0.17113973512501676), ('Alexa', -0.23074478358119999), ('Andy', 0.3352072778581667), ('James', -0.12350686241670664), ('Paul', -0.4411828690403207), ('Jason', 0.2604618505402568), ('Eric', 0.21033789465626948), ('Justin', 0.19768778356316113), ('Brian', -0.26744549649612553), ('Echo', 0.09759000729485331), ('Jake', -0.17964472806215853), ('Jeff', 0.31587698635699596), ('Lisa', 0.19181484192436687), ('Alex', 0.17155582447829892), ('Chuck', -0.286800242824307), ('Fred', -0.11023255019054999), ('J

In [181]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['UserID'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,UserID
0,-0.060163,Mike
1,0.143476,John
2,-0.085810,Chris
3,0.187140,Bill
4,0.006328,Dave


###### The top 50 similar users to input user

In [193]:
# top 50
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,UserID
65,0.577350,Charlie
90,0.556555,William
85,0.544682,Manny
64,0.518334,Tommy
45,0.474477,Jenn


###### Recommending products to the input user

In [183]:
topUsersRating=topUsers.merge(new, left_on='UserID', right_on='UserID', how='inner')
topUsersRating.head()

,similarityIndex,UserID,ProductId,name,rating
0,0.57735,Charlie,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",5.0
1,0.57735,Charlie,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",4.0
2,0.57735,Charlie,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",4.0
3,0.57735,Charlie,AVphgVaX1cnluZ0-DR74,"Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes...",5.0
4,0.57735,Charlie,AVphgVaX1cnluZ0-DR74,"Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes...",1.0


In [194]:
#Rating of selected users to all products
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,UserID,ProductId,name,rating,weightedRating
0,0.57735,Charlie,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",5.0,2.886751
1,0.57735,Charlie,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",4.0,2.309401
2,0.57735,Charlie,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",4.0,2.309401
3,0.57735,Charlie,AVphgVaX1cnluZ0-DR74,"Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes...",5.0,2.886751
4,0.57735,Charlie,AVphgVaX1cnluZ0-DR74,"Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes...",1.0,0.577350


In [185]:
tempTopUsersRating = topUsersRating.groupby('ProductId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
ProductId,,
AV1YE_muvKc47QAVgpwE,46.502480,226.771915
AV1YnR7wglJLPUi8IJmi,3.616124,16.449821
AV1YnRtnglJLPUi8IJmV,31.857373,154.172908
AVpfIfGA1cnluZ0-emyp,1.310362,5.462445
AVpf_znpilAPnD_xlvAF,0.252497,1.262485


In [195]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Taking the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['ProductId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,ProductId
ProductId,,
AV1YE_muvKc47QAVgpwE,4.876555,AV1YE_muvKc47QAVgpwE
AV1YnR7wglJLPUi8IJmi,4.549020,AV1YnR7wglJLPUi8IJmi
AV1YnRtnglJLPUi8IJmV,4.839473,AV1YnRtnglJLPUi8IJmV
AVpfIfGA1cnluZ0-emyp,4.168654,AVpfIfGA1cnluZ0-emyp
AVpf_znpilAPnD_xlvAF,5.000000,AVpf_znpilAPnD_xlvAF


###### Sorting out the weighted recommendation score to see the top10 mproducts that the algorithm has recommended

In [196]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)

In [197]:
recommended=new.loc[new['ProductId'].isin(recommendation_df.head(10)['ProductId'].tolist())]


In [198]:
recommended.drop_duplicates(subset=['ProductId'], inplace=True)

C:\Users\Stas\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# Top 10 Recommended Products

In [199]:
recommended[['ProductId', 'name']]

,ProductId,name
3067,AVqkIj9snnc1JgDc3khU,Amazon 5W USB Official OEM Charger and Power A...
14668,AVpftoij1cnluZ0-p5n2,Brand New Amazon Kindle Fire 16gb 7 Ips Displa...
14733,AVpjEN4jLJeJML43rpUe,Brand New Amazon Kindle Fire 16gb 7 Ips Displa...
17462,AV1YnRtnglJLPUi8IJmV,Amazon Kindle Paperwhite - eBook reader - 4 GB...
20640,AVphPmHuilAPnD_x3E5h,"Kindle Voyage E-reader, 6 High-Resolution Disp..."
21247,AVqkIdntQMlgsOJE6fuB,"Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes..."
21955,AVpfl8cLLJeJML43AE3S,"Echo (White),,,\r\nEcho (White),,,"
28936,AVpfpK8KLJeJML43BCuD,"Featured Brands,Electronics,Amazon Devices,Hom..."
29585,AV1YE_muvKc47QAVgpwE,"Back To College,College Electronics,College Tv..."
34635,AVpf_znpilAPnD_xlvAF,"Electronics,Amazon Device Accessories,Kindle S..."
